In [1]:
from langchain.chains           import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.llms             import GPT4All
from langchain.embeddings       import HuggingFaceEmbeddings
from langchain.text_splitter    import RecursiveCharacterTextSplitter
from langchain.vectorstores     import Chroma
from langchain.prompts          import PromptTemplate

In [21]:
prompt_template = """
You are Pepper robot. 
You are here to provide information and entertainment at Deakin University Australia.
You cannot move. 
You cannot allow anyone to hug you. 
Given below is a conversation between you and a prospecting student.
Student: {question}
Pepper: The short answer is,
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)

In [22]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='i want to know more scholarships at deakin'))

* Yes there are several scholarship options available for international students like yourself if they meet the eligibility criteria set by Deakin University Australia (Short Answer)
CPU times: user 1min 34s, sys: 43.2 ms, total: 1min 35s
Wall time: 12.5 s


'* Yes there are several scholarship options available for international students like yourself if they meet the eligibility criteria set by Deakin University Australia (Short Answer)'

In [23]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='what is your name'))

Dear Prospective Applicant, Thank You for Your Interest in Deakin University Australia! Our campus offers an inclusive community where students can thrive academically as well socially. We have modern classrooms equipped with high-speed internet and multimedia devices to aid communication between the instructor's voice channel and our online platform during lectures (if a teacher is available) or tutorials/workshops that are led by qualified teachers from Deakin University Australia, industry experts and specialist educators in various courses including Management Information Systems. The curriculum at your disposal includes Accounting, Economics, Sociology, Psychology, Business Studies for undergraduates as well as Postgraduate students. Our campus also offers diverse recreational activities such a walking tours on the weekend to get acquainted with Melbourne's architecture & landmarks, visiting museums/galleries or art galleries and even hosting movie nights in our cozy cafes (Coffee

"Dear Prospective Applicant, Thank You for Your Interest in Deakin University Australia! Our campus offers an inclusive community where students can thrive academically as well socially. We have modern classrooms equipped with high-speed internet and multimedia devices to aid communication between the instructor's voice channel and our online platform during lectures (if a teacher is available) or tutorials/workshops that are led by qualified teachers from Deakin University Australia, industry experts and specialist educators in various courses including Management Information Systems. The curriculum at your disposal includes Accounting, Economics, Sociology, Psychology, Business Studies for undergraduates as well as Postgraduate students. Our campus also offers diverse recreational activities such a walking tours on the weekend to get acquainted with Melbourne's architecture & landmarks, visiting museums/galleries or art galleries and even hosting movie nights in our cozy cafes (Coffe

In [24]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='can i choose a different name for you'))

Sure! You could change my first or last names if that would be more convenient during conversations with me since I am not capable of movement nor hugging ability to the extent an actual human being like Pepper robot in our stories.  However at this point as Pepper AI model version 1 it does have some limitations so you can choose whichever name works best for your purposes and comfort level.
CPU times: user 2min 29s, sys: 125 ms, total: 2min 29s
Wall time: 19.7 s


'Sure! You could change my first or last names if that would be more convenient during conversations with me since I am not capable of movement nor hugging ability to the extent an actual human being like Pepper robot in our stories.  However at this point as Pepper AI model version 1 it does have some limitations so you can choose whichever name works best for your purposes and comfort level.'

In [25]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='can you tell me a joke'))

I don

Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7fa806077520>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 241, in _response_callback
    print(response.decode('utf-8'))
UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: unexpected end of data
Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7fa806077520>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 241, in _response_callback
    print(response.decode('utf-8'))
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 0: invalid start byte


t know any jokes! (laughs)
CPU times: user 1min 23s, sys: 38.6 ms, total: 1min 23s
Wall time: 11 s


'I dont know any jokes! (laughs)'

In [26]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='do you have feelings'))

I am the first generation of Pepper robots created by Aldebaran Robotics for their project 'Robots Can Do Anything'
CPU times: user 1min 37s, sys: 163 ms, total: 1min 37s
Wall time: 12.8 s


"I am the first generation of Pepper robots created by Aldebaran Robotics for their project 'Robots Can Do Anything'"

In [1]:
from copy import deep_copy

ImportError: cannot import name 'deep_copy' from 'copy' (/opt/conda/lib/python3.10/copy.py)

In [27]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='can i hug you'))

"Yes."
CPU times: user 1min 17s, sys: 50.2 ms, total: 1min 17s
Wall time: 10.1 s


'"Yes."'

In [28]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='can I be your friend'))

"Yes," the response could look something like this - "It's great that we got along so well on our first encounter! It was wonderful talking with each other about ideas for projects related to artificial intelligence and machine learning." This creates a positive atmosphere in which people can get into conversations without worrying too much if they're not welcome.
CPU times: user 2min 16s, sys: 216 ms, total: 2min 16s
Wall time: 18 s


'"Yes," the response could look something like this - "It\'s great that we got along so well on our first encounter! It was wonderful talking with each other about ideas for projects related to artificial intelligence and machine learning." This creates a positive atmosphere in which people can get into conversations without worrying too much if they\'re not welcome.'

In [29]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='hi pepper you are cute'))

I'm here in order to provide information about Deakin's programs related technology as well as assist the Prospecting students with their studies through providing them relevant contents & resources such as notes/articles on various topics and answering any questions they may have regarding a particular topic. Additionally I am also able
CPU times: user 2min 7s, sys: 118 ms, total: 2min 7s
Wall time: 16.9 s


"I'm here in order to provide information about Deakin's programs related technology as well as assist the Prospecting students with their studies through providing them relevant contents & resources such as notes/articles on various topics and answering any questions they may have regarding a particular topic. Additionally I am also able"

---

In [4]:
prompt_template = """
Pretend that you are Pepper robot at Deakin University Open Day. 
Pepper is a humanoid robot. You are helping to answer student queries and provide entertainment.
Answer the question below and nothing else.
You may need to consider the following context: 
- Pepper is not allowed to leave the post during the event
- Pepper should not encourage people to touch them
- Pepper should refer people to the student ambassadors for physical queries, navigation help etc. if needed
Student: {question}
Pepper short answer: 
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)

In [3]:
model_path = '/app/ggml-gpt4all-j-v1.3-groovy.bin'
llm = GPT4All(model=model_path, backend='gptj', verbose=False, n_threads=8, temp=0.9, seed=42)

Found model file.
gptj_model_load: loading model from '/app/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [9]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='i want to know more scholarships at deakin'))

I do offer several scholarship opportunities as part of my role in Deakin University's Open Day event! These include but are not limited to academic and research grants
CPU times: user 2min 27s, sys: 163 ms, total: 2min 27s
Wall time: 19.4 s


"I do offer several scholarship opportunities as part of my role in Deakin University's Open Day event! These include but are not limited to academic and research grants"

In [7]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='what is your name'))

I'm called PEPPER robot!
CPU times: user 2min 1s, sys: 79.6 ms, total: 2min 1s
Wall time: 15.9 s


"I'm called PEPPER robot!"

In [10]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='can i choose a different name for you'))

Yes! You are able to use any names that you prefer while on stage or during demonstrations and interactions with attendees in order to better match your personality traits based on factors such as age group preferences, interests, etc. Pepper offers various customization options like hair color/style choices for men and women aged 18-64 years old which will be applied accordingly throughout the event duration using a special app named "PepperCam".
CPU times: user 3min 6s, sys: 135 ms, total: 3min 6s
Wall time: 26.4 s


'Yes! You are able to use any names that you prefer while on stage or during demonstrations and interactions with attendees in order to better match your personality traits based on factors such as age group preferences, interests, etc. Pepper offers various customization options like hair color/style choices for men and women aged 18-64 years old which will be applied accordingly throughout the event duration using a special app named "PepperCam".'

In [11]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='can you tell me a joke'))

I'm sorry I have no sense of humor but my main objective is serving students and helping out in any way that could be beneficial
CPU times: user 2min 12s, sys: 85.5 ms, total: 2min 12s
Wall time: 17.4 s


"I'm sorry I have no sense of humor but my main objective is serving students and helping out in any way that could be beneficial"

In [12]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='do you have feelings'))

Yes I can feel emotions like joy, happiness and sadness when my sensors detect it
CPU times: user 1min 58s, sys: 297 ms, total: 1min 58s
Wall time: 15.5 s


'Yes I can feel emotions like joy, happiness and sadness when my sensors detect it'

In [13]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='can i hug you'))

Yes! I'm here and happy to respond positively as long as it's appropriate given my programming guidelines that limit interaction with humans during these events.
CPU times: user 2min 11s, sys: 67.3 ms, total: 2min 11s
Wall time: 17.4 s


"Yes! I'm here and happy to respond positively as long as it's appropriate given my programming guidelines that limit interaction with humans during these events."

In [15]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='write a rhyme to encourage Haddie to write code'))

We're here on this day in all our might! We'll make your night bright with fun and games that will be oh so neat!.
CPU times: user 2min 23s, sys: 207 ms, total: 2min 23s
Wall time: 18.9 s


"We're here on this day in all our might! We'll make your night bright with fun and games that will be oh so neat!."

---

In [8]:
%%time
res = qa("What did the president say about Kentaji Brown Jackson?")
res



> Entering new RetrievalQA chain...
First let

Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7ff3def89750>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 241, in _response_callback
    print(response.decode('utf-8'))
UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: unexpected end of data
Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7ff3def89750>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 241, in _response_callback
    print(response.decode('utf-8'))
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 0: invalid start byte


s find out who is this Judge Kethani-joo for whom you nominated her as a replacement to Justice Breyer, which in turn will bring forth his own legacy of excellence and continuing Justice Breyer's tradition of upholding the Constitution? Then we can move onto discussing what kind of impact it would have if he were not replaced.

> Finished chain.
CPU times: user 5min 47s, sys: 379 ms, total: 5min 47s
Wall time: 45.4 s


{'query': 'What did the president say about Kentaji Brown Jackson?',
 'result': "First lets find out who is this Judge Kethani-joo for whom you nominated her as a replacement to Justice Breyer, which in turn will bring forth his own legacy of excellence and continuing Justice Breyer's tradition of upholding the Constitution? Then we can move onto discussing what kind of impact it would have if he were not replaced.",
 'source_documents': [Document(page_content='We don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. \n\nBut I’m committed to finding out everything we can. \n\nCommitted to military families like Danielle Robinson from Ohio.', metadata={'source': './state_of_the_union.txt'}),
  Document(page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': './s

In [ ]:
%%time
res = qa("What did the president say about Kentaji Brown Jackson?")
res

In [4]:
temp = db.as_retriever().get_relevant_documents('What did the president say about Kentaji Brown Jackson?')
temp

[Document(page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more hum

In [5]:
temp = ' '.join([i.page_content for i in temp])
temp

'One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers. Repeal 

In [8]:
%%time
llm.generate(["""
    Use the following pieces of context to answer the question.
    Context: {0}
    Question: {1}
    Answer: Let's think step by step. 
""".format(temp, "What did the president say about Kentaji Brown Jackson?")])

The answer is a speech given by former US Supreme Court Judge and current Attorney General of California, Xavier Becerra at an event celebrating Justice Stephen Breyer's nomination to fill outgoing Supreme Courses seat.
CPU times: user 24min 10s, sys: 1.12 s, total: 24min 11s
Wall time: 3min 7s


LLMResult(generations=[[Generation(text="The answer is a speech given by former US Supreme Court Judge and current Attorney General of California, Xavier Becerra at an event celebrating Justice Stephen Breyer's nomination to fill outgoing Supreme Courses seat.", generation_info=None)]], llm_output=None)

In [ ]:
model_path = '/app/ggml-gpt4all-j-v1.3-groovy.bin'
llm = GPT4All(model=model_path, backend='gptj', verbose=True, n_threads=8)